In [1]:
import sys
sys.path.insert(1, '/home/linus/projects/fdl/crisp')

In [2]:
%pwd

'/home/jupyter/crisp/notebooks'

In [3]:
import os
import pandas as pd
os.chdir("..")

In [4]:
from models.Deconfounder import *
from models.TorchLinearInvariantCausalPrediction import TorchInvariantCausalPrediction
from dataio.DataFrameDataset import *
from dataio.datasets import *

In [12]:
from synthetic.facebook_synthetic_data_generator import generator_example
from synthetic.synthetic_generator_conf import synthetic_generator_counfounder, synthetic_generator_nonlinear_environments
from synthetic.synthetic_generator import synthetic_generator

n_example = 2
dim_inv=2
n_bin=1
dim_spu=2
n_exp_train = 1000
n_exp_test = 400
n_exp = n_exp_train
n_env=2
save_dir= 'data/synthetic'
test=False 



In [6]:
config = {
    "name": "Example Experiment for AH casual ensemble",
    "short_name": "ah_experiment_notebook",
    "bucket_project": "fdl-us-astronaut-health",
    "bucket_name": "ah_21_data",
    "bucket_path": "gs://ah_21_data",
    "verbose": 1,
    "test_val_split": [0.1, 0.1],
    "per_variant_experiment": False,
    "data_options": {
        #'dataset_fp': '../data/test_multiclass.pkl',
        'dataset_fp' : None,
        'output_data_regime' : ["real-valued", "binary", "binary", "real-valued", "multi-class", "real-valued"],
        'subject_keys': 'Subj_ID',
        'targets': ['Target'],
 #        'predictors': ['All'],
        'predictors': None,
        'environments': ['env_split'],
        'exclude': ['Subj_ID']
    },
    "feature_selection_options": {
        "max_features": 20,
        "verbose": 0,
        "seed": 12
    },
    "ensemble_options": {
        "models": ["ERM", "RF", "ICP", "IRM", "DCF", "ITE", "LIRM", "NLICP"]
    },
    "use_cloud":False,
    "results_directory": "results/"
}
data_config = config['data_options']

In [7]:
icp_args = {
    "minP": 0.1,
    "maxP": 0.9,
    "minFeatures": 1,
    "minAccuracy": 0.5,
    "seed": 0,
    "verbose": 1,
    "target": data_config['targets'],
    "output_pvals": False # we cannot output pvalues in torch on the current build 
}

## Testing on the FB datasets

In [8]:
results = {}

for n_example in [1,2,3,5]:
    print("\n\nn_example:", n_example, "\n\n")
    if n_example == 4:
        dim_inv=(2,2)
        dim_spu=(2,2)
        
    generator_example(n_example, dim_inv, dim_spu, n_exp, n_env ,save_dir, test)
    
    if n_example == 4:
        dim_inv=4
        dim_spu=4
        
    outfile = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp, n_env,test))
    if n_example == 6:
        outfile = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s_n_bin_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp, n_env,test,n_bin))
    df_train = pd.read_pickle(outfile)
    df_train = df_train[df_train["env_split"]==0]
    
    #df_train = df_train.drop(columns=["env_split", "Subj_ID"])
    #df_train.head()
    config["data_options"]["dataset_fp"] = outfile
    config["data_options"]["predictors"] = list(df_train.columns[0:dim_inv+dim_spu])
    environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

    icp_args["target"] = config["data_options"]["targets"]
    icp_args["output_data_regime"] = config["data_options"]["output_data_regime"][n_example-1]
    icp_args["columns"] = test_dataset.predictor_columns

    icp = TorchInvariantCausalPrediction(environment_datasets, val_dataset, test_dataset, icp_args)
    icp_results_dict = icp.results()
    
    results[n_example] = icp_results_dict



n_example: 1 


Environments variables: {'E0': 0.1, 'E1': 1.5}
Generated Synthetic Data according to the Facebook setup Example: 1
Running a per sample experiment
Loaded  2  train environments
Env  0  has  417  samples
X shape  (417, 4)  y shape  (417, 1)
Env  1  has  393  samples
X shape  (393, 4)  y shape  (393, 1)
Loaded val set, X shape: (90, 4)  y shape:  (90, 1)
Loaded test set, X shape: (100, 4)  y shape:  (100, 1)
Testing  16  permutations with max set size:  4
Rejected subset 0
No accepted sets found, please consider decreasing {alpha} or try non-linear ICP


n_example: 2 


Environments variables: {'E0': {'p': 0.95, 's': 0.3}, 'E1': {'p': 0.97, 's': 0.5}}
Generated Synthetic Data according to the Facebook setup Example: 2
Running a per sample experiment
Loaded  2  train environments
Env  0  has  406  samples
X shape  (406, 4)  y shape  (406, 1)
Env  1  has  404  samples
X shape  (404, 4)  y shape  (404, 1)
Loaded val set, X shape: (90, 4)  y shape:  (90, 1)
Loaded test set,

## Testing on the CRSIP data: Confounder, and nonlinear dynamics

In [11]:
print("\n\nn_example: Confounder \n\n")

df_train = synthetic_generator_counfounder(n=1000,d_layer=3,n_layer=[4,2,2],mu=0,sigma=1,n_causal=4, n_env=5)
synthetic_name = 'full_fw_synthetic__counf_sem_n_causal_'+str(4)+'.pickle'
synthetic_loc = 'data/synthetic/'+synthetic_name
df_train.to_pickle(synthetic_loc)
        
outfile = synthetic_loc #os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp, n_env,test))

config["data_options"]["dataset_fp"] = outfile
config["data_options"]["predictors"] = list(df_train.columns[1:dim_inv+dim_spu])
environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

icp_args["target"] = config["data_options"]["targets"]
icp_args["output_data_regime"] = config["data_options"]["output_data_regime"][n_example-1]
icp_args["columns"] = test_dataset.predictor_columns

icp = TorchInvariantCausalPrediction(environment_datasets, val_dataset, test_dataset, icp_args)
icp_results_dict = icp.results()

results[n_example] = icp_results_dict



n_example: Confounder 


Running a per sample experiment
Loaded  5  train environments
Env  0  has  152  samples
X shape  (152, 3)  y shape  (152, 1)
Env  1  has  166  samples
X shape  (166, 3)  y shape  (166, 1)
Env  2  has  164  samples
X shape  (164, 3)  y shape  (164, 1)
Env  3  has  167  samples
X shape  (167, 3)  y shape  (167, 1)
Env  4  has  161  samples
X shape  (161, 3)  y shape  (161, 1)
Loaded val set, X shape: (90, 3)  y shape:  (90, 1)
Loaded test set, X shape: (100, 3)  y shape:  (100, 1)
Testing  8  permutations with max set size:  3
Rejected subset 0
No accepted sets found, please consider decreasing {alpha} or try non-linear ICP


In [14]:
print("\n\nn_example: Nonlinear \n\n")

df_train = synthetic_generator_nonlinear_environments(n=100,d_layer=3,n_layer=[4,2,2],mu=0,sigma=1,n_causal=4)
synthetic_name = 'full_fw_synthetic__counf_sem_n_causal_'+str(4)+'.pickle'
synthetic_loc = 'data/synthetic/'+synthetic_name
df_train.to_pickle(synthetic_loc)
        
outfile = synthetic_loc #os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp, n_env,test))

config["data_options"]["dataset_fp"] = outfile
config["data_options"]["predictors"] = list(df_train.columns[1:dim_inv+dim_spu])
environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

icp_args["target"] = config["data_options"]["targets"]
icp_args["output_data_regime"] = config["data_options"]["output_data_regime"][n_example-1]
icp_args["columns"] = test_dataset.predictor_columns

icp = TorchInvariantCausalPrediction(environment_datasets, val_dataset, test_dataset, icp_args)
icp_results_dict = icp.results()

results[n_example] = icp_results_dict



n_example: Nonlinear 


Running a per sample experiment
Loaded  2  train environments
Env  0  has  38  samples
X shape  (38, 3)  y shape  (38, 1)
Env  1  has  45  samples
X shape  (45, 3)  y shape  (45, 1)
Loaded val set, X shape: (7, 3)  y shape:  (7, 1)
Loaded test set, X shape: (10, 3)  y shape:  (10, 1)
Testing  8  permutations with max set size:  3
Accepted subset: (0,) ['Causal_0']
Accepted subset: (1,) ['Causal_1']
Accepted subset: (2,) ['Causal_2']
Accepted subset: (0, 1) ['Causal_0' 'Causal_1']
Accepted subset: (0, 2) ['Causal_0' 'Causal_2']
Accepted subset: (1, 2) ['Causal_1' 'Causal_2']
Accepted subset: (0, 1, 2) ['Causal_0' 'Causal_1' 'Causal_2']
Intersection: []
No intersection, trying defining sets
Pruning defining set trees
Found the defining sets!
Defining Sets: [[1, 2, 0]]


In [9]:
df_train.Target.unique()

array([-2.14994118e-01,  6.76648226e-03,  1.68722525e-01, -1.75363213e-01,
        3.98466736e-03,  4.98066843e-03, -6.85107633e-02,  2.01874122e-01,
       -2.16742814e-01,  1.68156564e-01, -5.03021069e-02,  6.26489669e-02,
       -2.36047059e-03, -1.12062342e-01, -3.44122797e-01, -2.20517889e-02,
        4.27747816e-02,  3.37011814e-02, -5.91753870e-02, -1.15692914e-01,
       -1.50741935e-01, -2.83219814e-01, -1.97393954e-01,  1.21190570e-01,
        1.25205845e-01, -1.86740570e-02, -3.46683443e-01, -1.82748616e-01,
       -2.21790925e-01,  8.63333195e-02, -1.10394970e-01,  2.18093842e-02,
       -5.96779063e-02,  1.67705461e-01,  2.72384197e-01, -1.52346909e-01,
       -6.01685643e-02,  1.57499611e-02,  5.05444184e-02,  1.30187154e-01,
        4.19126362e-01, -1.74650893e-01,  2.74098925e-02, -1.94627076e-01,
       -1.85011566e-01,  1.83176190e-01, -2.24702954e-02,  1.38719693e-01,
       -1.84895694e-02, -4.03451733e-02,  1.18836276e-02,  1.57602936e-01,
       -2.03211263e-01,  

In [10]:
results

{1: {'solution': False,
  'intersection': False,
  'test_acc': 0,
  'test_nll': 1000000000.0,
  'test_probs': None,
  'test_labels': None,
  'feature_coeffients': tensor([0., 0., 0., 0.]),
  'selected_features': None,
  'selected_feature_indices': None,
  'to_bucket': {'method': 'Linear ICP',
   'features': None,
   'coefficients': None,
   'pvals': None,
   'test_acc': None,
   'test_acc_std': None}},
 2: {'solution': False,
  'intersection': False,
  'test_acc': 0,
  'test_nll': 1000000000.0,
  'test_probs': None,
  'test_labels': None,
  'feature_coeffients': tensor([0., 0., 0., 0.]),
  'selected_features': None,
  'selected_feature_indices': None,
  'to_bucket': {'method': 'Linear ICP',
   'features': None,
   'coefficients': None,
   'pvals': None,
   'test_acc': None,
   'test_acc_std': None}},
 3: {'solution': True,
  'intersection': False,
  'test_acc': 0.5,
  'test_nll': tensor(0.6931),
  'test_probs': tensor([[0.5339, 0.4661],
          [0.4969, 0.5031],
          [0.4965, 0.

In [11]:
#from synthetic.synthetic_generator import synthetic_generator
#df = synthetic_generator()
#df["Target"] = np.random.randint(0,3,size=df.shape[0])
#df.to_pickle("../data/test_multiclass.pkl")

In [12]:
import pandas as pd
import pickle
#with open("../data/test.pkl",'rb') as file:
with open("../data/test_multiclass.pkl",'rb') as file:
    df = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '../data/test_multiclass.pkl'

In [ ]:
environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

In [ ]:
# run DCF
dcf_args = {
    "minP": 0.1,
    "maxP": 0.9,
    "minFeatures": 1,
    "minAccuracy": 0.5,
    "seed": 0,
    "verbose": 1,
    "target": data_config['targets'],
    "output_pvals": False # we cannot output pvalues in torch on the current build 
}
dcf_args["columns"] = test_dataset.predictor_columns

dcf = TorchMultiClassDeconfounder(environment_datasets, val_dataset, test_dataset, dcf_args)

dcf_results_dict = dcf.predictor_results()
print("Finished DCF")
print(dcf_results_dict)

In [ ]:
dcf_results_dict["results"]["Features"]

In [ ]:
dcf_results_dict["results"]["Features"]

In [ ]:
# run DCF
dcf_args = {
    "minP": 0.1,
    "maxP": 0.9,
    "minFeatures": 1,
    "minAccuracy": 0.5,
    "seed": 0,
    "verbose": 1,
    "target": data_config['targets'],
    "output_pvals": False # we cannot output pvalues in torch on the current build 
}
dcf_args["columns"] = test_dataset.predictor_columns

dcf = Deconfounder(environment_datasets, val_dataset, test_dataset, dcf_args)

dcf_results_dict = dcf.predictor_results()
print("Finished DCF")
print(dcf_results_dict)

In [ ]:
torch_ICP_results_dict['selected_features']